In [ ]:
#import all necessary python extensions#
import os
from zipfile import ZipFile
import io
import csv
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np
import glob

In [ ]:
#extract all zipfiles to temporary folder#
zip_files = os.listdir("Zipfiles/")
for item in os.listdir("Zipfiles/"):
    if item.endswith(".zip"):
        with ZipFile("Zipfiles/" + item, 'r') as zf:
            zf.printdir()
            zf.extractall("TempZipFolder/")



In [ ]:
#find out which files are semicolon delimited#
delimiters = dict() 
for item in os.listdir("Zipfiles/"):
    if item.endswith(".zip"):
        with ZipFile("Zipfiles/" + item, 'r') as zf:
            zipped_files = zf.namelist()
            for file_name in zipped_files :
                input_file = zf.open(file_name,'r')
                input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
                dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
                delimiters[file_name] = dialect.delimiter
            
                #input_file.seek(0)
            
                print(" ".join(["It looks like",
                           file_name,
                           "has delimiter",
                           dialect.delimiter,
                           "."]))

                input_file.close() # tidy up

In [ ]:
#create a list of semicolon_delimited files for import#
semicolon_delimited = [k for k,v in delimiters.items() if v == ';']
comma_delimited = [k for k,v in delimiters.items() if v == ',']
print(semicolon_delimited[1])

In [ ]:
#find which files have headers, make a list of those filenames#
noheads = []
for this_zf in zip_files :
    with ZipFile("ZipFiles/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            this_delimiter = delimiters[file_name]
            
            for line in input_file :
                    noheads.append(file_name + ';' + str("datetime" in line))
                    break
            input_file.close() # tidy up
heads = []
for obj in noheads:
    if obj[-4:] == r'True':
        heads.append(obj.split(';')[0])

In [ ]:
#using pandas, clean files of commas, semicolons, and line breaks, export all back to csv with same headers and commas#
for obj in os.listdir("CleanFiles/"):
    if obj.endswith(".csv"):
        os.remove("CleanFiles/" + obj)
for part in os.listdir("TempZipFolder/"):
    headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]
    for ele in semicolon_delimited:
        if part == ele:
            df = pd.read_csv("TempZipFolder/" + part, sep = ';', header=0, names=headers, encoding = 'utf-8',low_memory=False)
            print(part + 'has semicolons')
            df = df.replace(r'\n'," ", regex=True)
            df = df.replace(r'\\n'," ", regex=True)
            df = df.replace(r'\N'," ")
            df = df.replace(r'\\N'," ")
            df = df.replace(r','," ", regex=True)
            df = df.replace(r';'," ", regex=True)
            df.to_csv(r'TempFiles/clean' + part, sep = ",", header = True, index = False)
            print("generated clean file for {}".format(part))
            break
    else:
        df = pd.read_csv("TempZipFolder/" + part, sep = ',', header=0, encoding = 'utf-8', names=headers,low_memory=False)
        df = df.replace(r'\n'," ", regex=True)
        df = df.replace(r'\\n'," ", regex=True)
        df = df.replace(r'\N'," ")
        df = df.replace(r'\\N'," ")
        df = df.replace(r','," ", regex=True)
        df = df.replace(r';'," ", regex=True)
        df.to_csv(r'TempFiles/clean' + part, sep = ",", header = True, index = False)
        print("generated clean file for {}".format(part))
        continue
        print('clean file upload complete')

In [ ]:
#reimport as DataFrame, this time to make all datatypes the same and remove population of "nan" values#
for part in os.listdir("TempFiles/"):
    for ele in os.listdir("CleanFiles/"):
        if part == ele:
            break
    else:
        df = pd.read_csv("TempFiles/" + part, sep = ',', header=0, names=headers, na_filter=False)
        df = df.astype({"datetime":"object","register_no":"float","emp_no":"float","trans_no":"float","upc":"string","description":"string","trans_type":"string","trans_subtype":"string","trans_status":"string","department":"float","quantity":"float","Scale":"float","cost":"float","unitPrice":"float","total":"float","regPrice":"float","altPrice":"float","tax":"float","taxexempt":"float","foodstamp":"float","wicable":"float","discount":"float","memDiscount":"float","discountable":"float","discounttype":"float","voided":"float","percentDiscount":"float","ItemQtty":"float","volDiscType":"float","volume":"float","VolSpecial":"float","mixMatch":"float","matched":"float","memType":"bool","staff":"bool","numflag":"float","itemstatus":"float","tenderstatus":"float","charflag":"string","varflag":"float","batchHeaderID":"bool","local":"float","organic":"float","display":"bool","receipt":"float","card_no":"float","store":"float","branch":"float","match_id":"float","trans_id":"float"},errors='ignore')
        df.to_csv(r'CleanFiles/' + part, sep = ",", header = True, index = False)
        print("generated clean file for {}".format(part))

In [ ]:
#create credentials, service file path, client for GBQ# 
service_path = "/Users/peterkirgis/Downloads/"
service_file = 'wedge-project-peterkirgis-1e7b528def57.json'
gbq_proj_id = 'wedge-project-peterkirgis'  
gbq_dataset_id = 'WedgeData2' 

private_key =service_path + service_file

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
#upload cleaned files to GBQ with schema#
for file in os.listdir('CleanFiles/'):
    if file.endswith('.csv'):
        tablename = file.split(".")[0]
        table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,tablename])
        print(table_full_name)
    
        def tbl_exists(client, table_ref):
            from google.cloud.exceptions import NotFound
            try:
                client.get_table(table_ref)
                return True
            except NotFound:
                return False
    
        if not tbl_exists(client, table_full_name) :
            table_ref = client.create_table(
            table = table_full_name
            )
        else :
            print("table {} already exists".format(table_full_name))
            continue
        
        job_config = bigquery.LoadJobConfig()
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
        job_config.schema_update_options = [
        bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION # This allows us to modify the table. 
        ]
        job_config.schema = [
        bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
        ]
        job_config.source_format = bigquery.SourceFormat.CSV
        job_config.skip_leading_rows = 1
    
        with open('CleanFiles/' + file, "rb") as source_file:
            job = client.load_table_from_file(
            source_file,
            table_ref,
            location="US",  # Must match the destination dataset location.
            job_config=job_config,
            )  # API request
        job.result()  # Waits for table load to complete.
        print(
        "Loaded {} rows into {}:{}.".format(
            job.output_rows, 'WedgeData2', table_ref.table_id))